<!-- Autoheader begin -->
<hr/>
<div id="navtitle_3_3_jl" style="text-align:center; font-size:16px">III.3 Using Krotov's method to separate chiral molecules</div>
<hr/>
<table style="width: 100%">
  <tr>
    <th rowspan="2" style="width:33%; text-align:center; font-size:16px">
        <a href="jl_exercise_3_2_lambda.ipynb">$\leftarrow$ previous notebook </a><br>
        <a href="jl_exercise_3_2_lambda.ipynb" style="font-size:13px">III.2 Optimal Control for STIRAP</a>
    </th>
    <td style="width:33%; text-align:center; font-size:16px">
        <a href="jl_exercise_2_3_chiral.ipynb">$\uparrow$ previous part $\uparrow$</a><br>
        <a href="jl_exercise_2_3_chiral.ipynb" style="font-size:13px">II.3 Parameter Optimization of Three-Wave Mixing in a Three-Level System</a>
    </td>
    <th rowspan="2" style="width:33%; text-align:center; font-size:16px">
        <a href="jl_exercise_3_4_gate.ipynb">next notebook $\rightarrow$</a><br>
        <a href="jl_exercise_3_4_gate.ipynb" style="font-size:13px">III.4 Entangling Quantum Gates for Coupled Transmon Qubits</a>
    </th>
  </tr>
  <tr style="width: 100%">
    <td style="width:33%; text-align:center; font-size:16px">
    </td>
  </tr>
</table>

<div style="text-align: right;font-size: 16px"><a href="../Python/py_exercise_3_3_chiral.ipynb">👉 Python version</a></div>

---
<!-- Autoheader end -->

# Using Krotov's method to separate chiral molecules

$
\newcommand{tr}[0]{\operatorname{tr}}
\newcommand{diag}[0]{\operatorname{diag}}
\newcommand{abs}[0]{\operatorname{abs}}
\newcommand{pop}[0]{\operatorname{pop}}
\newcommand{aux}[0]{\text{aux}}
\newcommand{opt}[0]{\text{opt}}
\newcommand{tgt}[0]{\text{tgt}}
\newcommand{init}[0]{\text{init}}
\newcommand{lab}[0]{\text{lab}}
\newcommand{rwa}[0]{\text{rwa}}
\newcommand{bra}[1]{\langle#1\vert}
\newcommand{ket}[1]{\vert#1\rangle}
\newcommand{Bra}[1]{\left\langle#1\right\vert}
\newcommand{Ket}[1]{\left\vert#1\right\rangle}
\newcommand{Braket}[2]{\left\langle #1\vphantom{#2} \mid #2\vphantom{#1}\right\rangle}
\newcommand{Ketbra}[2]{\left\vert#1\vphantom{#2} \right\rangle \hspace{-0.2em} \left\langle #2\vphantom{#1}\right\vert}
\newcommand{e}[1]{\mathrm{e}^{#1}}
\newcommand{op}[1]{\hat{#1}}
\newcommand{Op}[1]{\hat{#1}}
\newcommand{dd}[0]{\,\text{d}}
\newcommand{Liouville}[0]{\mathcal{L}}
\newcommand{DynMap}[0]{\mathcal{E}}
\newcommand{identity}[0]{\mathbf{1}}
\newcommand{Norm}[1]{\lVert#1\rVert}
\newcommand{Abs}[1]{\left\vert#1\right\vert}
\newcommand{avg}[1]{\langle#1\rangle}
\newcommand{Avg}[1]{\left\langle#1\right\rangle}
\newcommand{AbsSq}[1]{\left\vert#1\right\vert^2}
\newcommand{Re}[0]{\operatorname{Re}}
\newcommand{Im}[0]{\operatorname{Im}}
\newcommand{toP}[0]{\omega_{12}}
\newcommand{toS}[0]{\omega_{23}}
\newcommand{oft}[0]{\left(t\right)}
$

In this notebook, you will run a gradient-based
optimization for three-wave mixing in a chiral three-level system,
building up on [Exercise II.3](jl_exercise_2_3_chiral.ipynb).
We keep the same goal of driving two three-level systems,
describing the quantum states of the enantiomers of a chiral molecule,
such that they end up in distinct final states, but use a gradient-based optimization method instead, specifically Krotov's method.

## Setup

First, we load some of the libraries which will be needed throughout this notebook.

In [ ]:
using LinearAlgebra
using QuantumControl
using Krotov

In [ ]:
const 𝕚 = 1im;

For visualization, we use the `Plots` package

In [ ]:
using Plots

# Set up thicker default lines in plots
Plots.default(
    linewidth               = 2.0,
    foreground_color_legend = nothing,
    background_color_legend = RGBA(1, 1, 1, 0.8)
)

In [ ]:
# Some utilities for showing hints and solutions
include(joinpath("utils", "exercise_3_chiral.jl"));

## Model

<center>
    <img src="../figures/3-level_mod.svg" alt="Drawing" width="750">
</center>

The system consists of three levels which evolve, using the rotating wave approximation, according to the relations
$$
\begin{align}
    i \frac{\partial}{\partial t} \Ket{\Psi ^{\,(\pm)} (t)}&= \sum_{i=1}^3 H_i ^{\,(\pm)} (t) \Ket{\Psi ^{\,(\pm)} (t)}\\
\Ket{\Psi ^{\,(\pm)} (t)} &= \sum_{i=1}^3 c_n^{\,(\pm)}(t)\, \Ket{n}.
\end{align}
$$

The different terms of the Hamiltonian can be expressed as

$$
\Op{H}_{1}^{\,(\pm)} = -E_1 (t)\, \,\mu_b^{\pm}\,\begin{pmatrix}
    0 & 1 & 0                          \\
    1 & 0 & 0 \\
    0 & 0 & 0
\end{pmatrix}\,.
$$

$$
\Op{H}_{2}^{\,(\pm)} = -E_2 (t)\, \,\mu_a^{\pm}\,\begin{pmatrix}
    0 & 0 & 0  \\
    0 & 0 & \mathrm{e}^{i\pi/2}  \\
    0 & \mathrm{e}^{-i\pi/2}  & 0
\end{pmatrix}\,.
$$

$$
\Op{H}_{3}^{\,(\pm)} = -E_3 (t)\, \,\mu_c^{\pm}\,\begin{pmatrix}
    0 & 0 & 1                          \\
    0 & 0 & 0 \\
    1 & 0 & 0
\end{pmatrix}\,,
$$

where $E_1(t)$, $E_2(t)$ and $E_3(t)$ are the envelopes of fields with central frequencies $\omega$, $\delta\omega$ and $\omega + \delta\omega$, respectively.
The interesting part of the dynamics comes from the relations between the dipole moments of the two enantiomers, namely $\mu_a^{\,(+)} =  \mu_a^{\,(-)}$, $\mu_b^{\,(+)} =  \mu_b^{\,(-)}$ and $\mu_c^{\,(+)} =  -\mu_c^{\,(-)}$.

This implies that $\Op{H}_{1}^{\,(+)} = \Op{H}_{1}^{\,(-)}$, $\Op{H}_{2}^{\,(+)} = \Op{H}_{2}^{\,(-)}$ and $\Op{H}_{3}^{\,(+)} = -\Op{H}_{3}^{\,(-)}$.


Our goal will be to optimize pulses $E_1(t)$, $E_2(t)$ and $E_3(t)$ such that for the initial states $\Ket{\Psi ^{\,(+)} (0)} =: \Ket{\Psi_{\init}^+} = \Ket{1}$ and $\Ket{\Psi ^{\,(-)} (0)}  =: \Ket{\Psi_{\init}^-}  = \Ket{1}$ and a previously defined final time $T$ we respectively reach the target state $\Ket{\Psi ^{\,(+)} (T)}  =: \Ket{\Psi_{\tgt}^{\,+}} = e^{i \Phi_+}\Ket{2}$ and $\Ket{\Psi ^{\,(-)} (T)}  =: \Ket{\Psi_{\tgt}^{\,-}} = e^{i \Phi_-}\Ket{3}$ for some unspecified phases $ \Phi_+$ and $ \Phi_-$.
That is, although both enantiomers start in the same state, they end up in entirely different states after interaction with the laser fields. Since the phases $ \Phi_+$ and $ \Phi_-$ do not affect this physical optimization goal, it is sufficient to only consider the populations in the following.

First, we define the various dipole moments.

In [ ]:
μa =  2.0  #Note that μa := μa_p = μa_m, and similarly for b
μb =  3.0
μc_p = 0.5
μc_m = -0.5;

Next, we define the times at which the pulses start and stop in our initial guesses.

In [ ]:
u_1_start = 0.0; # as a factor of the overall time
u_1_stop  = 1.0; # as a factor of the overall time
u_2_start = 0.0; # as a factor of the overall time
u_2_stop  = 1.0; # as a factor of the overall time
u_3_start = 0.0; # as a factor of the overall time
u_3_stop  = 1.0; # as a factor of the overall time

... and the time grid

In [ ]:
T = 50
tlist = collect(range(0, T; length=501));

We choose an initial guess consisting of two low-intensity [Blackman
pulses](https://juliaquantumcontrol.github.io/QuantumControl.jl/stable/api/quantum_propagators/#QuantumPropagators.Shapes.blackman).

In [ ]:
using QuantumControl.Shapes: blackman

In [ ]:
"""Guess for the first electric pulse"""
u1(t) = .25 * blackman(t, u_1_start*T, u_1_stop*T);
"""Guess for the first electric pulse"""
u2(t) = .25 * blackman(t, u_2_start*T, u_2_stop*T);
"""Guess for the third electric pulse"""
u3(t) = .25 * blackman(t, u_3_start*T, u_3_stop*T);

Now we go on to define the Hamiltonians.

In [ ]:
"""3-level Chiral System Hamiltonian"""
function ham(; μa, μb, μc_p, μc_m)
    H1 = μb * [0.0 1.0 0.0; 1.0 0.0 0.0; 0.0 0.0 0.0]
    H2 = μa * [0.0 0.0 0.0; 0.0 0.0 1im; 0.0 -1im 0.0]

    H3_p = μc_p * [0.0 0.0 1.0; 0.0 0.0 0.0; 1.0 0.0 0.0]
    H3_m = μc_m * [0.0 0.0 1.0; 0.0 0.0 0.0; 1.0 0.0 0.0]

    H_p = hamiltonian((H1, u1), (H2, u2), (H3_p, u3))
    H_m = hamiltonian((H1, u1), (H2, u2), (H3_m, u3))
    return H_p, H_m
end

H_p, H_m = ham(; μa, μb, μc_p, μc_m);

The pulses look as follows:

In [ ]:
using QuantumControl.Controls: get_controls

In [ ]:
function plot_pulses(H, tlist)
    axs = []
    controls = get_controls(H)
    @assert length(controls) == 3
    titles = ["u₁(t)", "u₂(t)", "u₃(t)"]
    for (i, control) in  enumerate(controls)
        ax = plot(
            tlist, control;
            label="", xlabel="time", ylabel="pulse amplitude",
            title=titles[i]
        )
        push!(axs, ax)
    end
    plot(axs...; layout=(1,3), size=(1500,300))
end

In [ ]:
plot_pulses(H_p, tlist)

The goal of this example is a state to state transfer with
initial state
$\Ket{\Psi_{\init}^+} = \Ket{\Psi_{\init}^-} = \Ket{1}$ for both enantiomers and target states
$\Ket{\Psi_{\tgt}^{\,+}} = \Ket{2}$ and $\Ket{\Psi_{\tgt}^{\,-}} = \Ket{3}$ (up to a phase) at final time $T$.

In [ ]:
"""Initial and target states"""

psi_init  = ComplexF64[1.,0.,0.];
psi_tgt_p = ComplexF64[0.,1.,0.];
psi_tgt_m = ComplexF64[0.,0.,1.];

## Problem 1: Choice of pulse shapes

We have described the pulses using Blackman shapes instead of Gaussian curves, their main difference being that they start and end with an amplitude of 0. Why do you think this is more convenient, especially from an experimental point of view?

In [ ]:
# problem_1.solution

## Simulate dynamics of the guess field

After assuring ourselves that our guess pulses appear as expected, we propagate
the system using our guess.
To track the population in the relevant levels, we first define appropriate projectors $\op{P}_{i} =
\Ketbra{i}{i}$ on the subspaces:

In [ ]:
proj1 = diagm([1,0,0])
proj2 = diagm([0,1,0])
proj3 = diagm([0,0,1])

They will help us later to analyze the population in the different levels.

We also set so-called optimization objectives using
the function `Trajectory`, which contain all the
relevant information needed for the propagation:

In [ ]:
traj_p = Trajectory( psi_init, H_p; target_state=psi_tgt_p )
traj_m = Trajectory( psi_init, H_m; target_state=psi_tgt_m )

Having them set up, we can use the Chebychev propagator to obtain the dynamics over time. Since no optimization was done so far, we propagate with the guess pulses:

In [ ]:
using QuantumPropagators: Cheby

In [ ]:
res_pt_p, res_pt_m = propagate_trajectories([traj_p, traj_m], tlist; method=Cheby, storage=true, observables=[proj1, proj2, proj3]);

Now let's plot the population dynamics:

In [ ]:
"""Routine to represent population of different states"""
function plot_population_3lvl(res_pt; tlist, title="")
    fig = plot(; xlabel="time", ylabel="population", title=title)
    hline!(fig, [1.0], c=:black, lw=0.5)
    plot!(fig, tlist, abs.(res_pt[1,:]), label="|⟨Ψ(t)|1⟩|²")
    plot!(fig, tlist, abs.(res_pt[2,:]), label="|⟨Ψ(t)|2⟩|²")
    plot!(fig, tlist, abs.(res_pt[3,:]), label="|⟨Ψ(t)|3⟩|²")
    return fig
end

In [ ]:
plot_population_3lvl(res_pt_p; tlist, title="Enantiomer (+)")

What you see here is how the populations of the three levels for enantiomer (+) change during the interaction with the guess pulses.
Remember: Our aim is to get all the population to state 2 and none in state 1!

In [ ]:
plot_population_3lvl(res_pt_m; tlist, title="Enantiomer (-)")

And here are the populations for enantiomer (-). As you see, the final state looks very similar to the one of enantiomer (+). There's still work to do to change this!

## Problem 2: Final populations

Try to extract the populations at final time for the (+) enantimer and the (-) enantiomer.  Make sure to use the actual value from the results `res_pt_m` and `res_pt_p`.

In [ ]:
pop1_plus = #
pop2_plus = #
pop3_plus = #

In [ ]:
pop1_minus = #
pop2_minus = #
pop3_minus = #

In [ ]:
print(round.(real([[pop1_plus pop2_plus pop3_plus]; [pop1_minus pop2_minus pop3_minus]]); digits=5))

In [ ]:
# problem_2.hint

In [ ]:
# problem_2.solution

If you look closely, the
populations seem to be very similar between the two enantiomers.
We wanted a total transfer of population to state 2, which
means this guess pulse is really not good at all...

Next, we show how the optimization changes the pulses to reach the desired states.

## Optimization

#### Setting the pulse options

To initialize the optimization we need to provide some guess pulses. When we have no information over the system we can use generic Gaussian-like pulses and see how it affects the population.

On top of that, we need to provide a shape that will determinate which parts of the pulse can be changed. This can ensure, for example, a smooth start and end of the pulse.

In [ ]:
using QuantumControl.Shapes: flattop

In [ ]:
"""Scales Krotov method's update of the pulse value at the time t"""
S(t) = flattop.(t; T, t_rise=0.05*T, func=:sinsq)

We choose an appropriate update factor step size $\frac{1}{\lambda_a}$ for the problem at hand and
make sure Krotov's algorithm considers pulses which start and end with zero amplitude.

In [ ]:
# Changing λ will directly affect how fast the pulse change.

# If it is too big, the iterations will barely change any pulse.
# If it is too small, the pulses may change abruptly.

λₐ = 3.0

# Define how the pulses are allowed to change during each iteration
pulse_options = IdDict(
    ϵ => Dict(:lambda_a => λₐ, :update_shape => S,)
    for ϵ in get_controls(H_p)
)

#### Applying Krotov's method

We now supply Krotov's algorithm with all the information it needs to optimize,
consisting of the `trajectories` (maximize population for target states at $t_{f}$),
`pulse_options` (the initial shapes of our pulses and how they may be changed)
as well as the `prop_method` to use and the
maximum number of iterations to perform, `iter_stop`.

In [ ]:
using QuantumControl.Functionals # needed for J_T_ss

In [ ]:
problem = ControlProblem(
    [traj_p, traj_m],
    tlist;
    pulse_options,  # or, `lambda_a=λₐ, update_shape=S`
    J_T=J_T_ss,
    iter_stop=5,
    check_convergence=res -> begin
        ((res.J_T <= 1e-3) && (res.converged = true) && (res.message = "J_T < 10⁻³"))
    end,
    prop_method=Cheby,
);

After setting up the `ControlProblem`, we can optimze the problem

In [ ]:
oct_result = optimize(problem; method=Krotov)

---

Among all the columns of data shown during optimization the most important one is the second column, showing the functional value $J_T$ at the current iteration.
If your optimization stops without reaching the desired value of convergence, have a look at the `ΔJ_T` column which shows just the difference in the functional between two iterations.

After optimizing the pulse, we can define new Hamiltonians, which contain the optimized pulses. For this, we substitute the the old pulse with the new one.

In [ ]:
using QuantumControl.Controls: substitute

In [ ]:
H_p_opt = substitute(
    H_p,
    IdDict(
        ϵ => oct_result.optimized_controls[i]
        for (i, ϵ) in enumerate(get_controls(H_p))
    )
);
H_m_opt = substitute(
    H_m,
    IdDict(
        ϵ => oct_result.optimized_controls[i]
        for (i, ϵ) in enumerate(get_controls(H_m))
    )
);

We might have found pulse-shapes that fulfill our objective, but what do
they look like?

In [ ]:
plot_pulses(H_p_opt, tlist)

Once we have the optimized pulses, we can plot the dynamics for the initial states. For this, we first need to propagate the initial states with `H_p_opt` and `H_m_opt`, respectively.

In [ ]:
res_opt_p = propagate(psi_init, H_p_opt, tlist; method=Cheby, storage=true, observables=[proj1, proj2, proj3]);
plot_population_3lvl(res_opt_p; tlist, title="Enantiomer (+)")

In [ ]:
res_opt_m = propagate(psi_init, H_m_opt, tlist; method=Cheby, storage=true, observables=[proj1, proj2, proj3]);
plot_population_3lvl(res_opt_m; tlist, title="Enantiomer (+)")

The optimization seems to not quite match our expectations yet. Try to adjust the parameters to improve the convergence.

Remember: For enantiomer (+), only level 2 should populated and for enantiomer (-) only level 3.

## Problem 3: Refining λₐ

Try to adjust the $\lambda_a$ parameter such that you obtain a good result ($J_T < 10^{-3}$) in less than 5 optimization steps.

In [ ]:
# problem_3.hint

In [ ]:
# problem_3.solution

## Next steps

If you are also interested in gate optimization for quantum information applications, 
we invite you to have a look at [Exercise III.4](jl_exercise_3_4_gate.ipynb).

Alternatively, you may find further example beyond the scope of this tutorial as part of the
[`QuantumControl` documentation](https://juliaquantumcontrol.github.io/QuantumControlExamples.jl/stable/). This includes an [example for optimizing a quantum gate in an open quantum system](https://juliaquantumcontrol.github.io/QuantumControlExamples.jl/stable/examples/rho_3states/#Optimization-of-a-Dissipative-Quantum-Gate). You may also find the [examples from the documentation of the Python `krotov` package](https://qucontrol.github.io/krotov/v1.2.1/09_examples.html) instructive (all of which can easily be translated to Julia).

<!-- Autofooter begin -->

---

[⬆︎ jump to top](#navtitle_3_3_jl)
<!-- Autofooter end -->